### Importing Libraries

In [1]:

import requests
import json
import pandas as pd
import time, datetime

In [20]:
class Collector:
    def __init__(self, start_date, end_date, coin):
        self.start_date = start_date
        self.end_date = end_date
        self.coin = coin
        self.dataframe = None
        self.collector_status = False
        self.transform_status = False

    def collector(self):
        try:
            api_url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='{self.coin}'&@dataInicial='{self.start_date}'&@dataFinalCotacao='{self.end_date}'&$top=10000&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"
            response = requests.get(api_url)
            response.raise_for_status()
            
            if response.status_code == 200:
                data = response.json()
                df = pd.DataFrame(data)
                self.dataframe = pd.json_normalize(df['value'])
                self.collector_status = True
            else:
                print('Failed to get data from API.')
                
        except requests.exceptions.RequestException as e:
            print('Failed to get data from API. Error: ', e)

    def transform(self):
        try:
            datetime_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            self.dataframe['sk_data'] = pd.to_datetime(self.dataframe['dataHoraCotacao']).dt.strftime('%Y%m%d')
            
            drop_columns = ['dataHoraCotacao','tipoBoletim']
            self.dataframe = self.dataframe.drop(columns=drop_columns)

            df_buy = pd.DataFrame({'SK_DATA':self.dataframe['SK_DATA'],'INDICADOR':'Euro/Compra',
                            'VALOR_DIA':self.dataframe['cotacaoCompra'],'DTA_CADASTRO':datetime_now})

            df_sell = pd.DataFrame({'SK_DATA':self.dataframe['SK_DATA'],'INDICADOR':'Euro/Venda',
                            'VALOR_DIA':self.dataframe['cotacaoVenda'],'DTA_CADASTRO':datetime_now})

            df_final = pd.concat([df_buy, df_sell], ignore_index=False)
            self.dataframe = df_final.sort_values(by='SK_DATA') 
            self.transform_status = True

        except Exception as e:
            print('Error when applying transformations: ', e)    
        
    

In [38]:
api_url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='EUR'&@dataInicial='01-01-2015'&@dataFinalCotacao='01-01-2016'&$top=10000&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"
response = requests.get(api_url)
response.raise_for_status()
            
data = response.json()
df = pd.DataFrame(data)
df_final = pd.json_normalize(df['value'])


In [28]:
datetime_now = datetime.datetime.now()
datetime_now = datetime_now.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]


In [39]:
df_final

,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,3.2361,3.2379,2015-01-02 13:09:00.008,Fechamento
1,3.2340,3.2349,2015-01-05 13:05:39.045,Fechamento
2,3.2165,3.2182,2015-01-06 13:02:40.508,Fechamento
3,3.1673,3.1683,2015-01-07 13:03:26.355,Fechamento
4,3.1749,3.1764,2015-01-08 13:14:50.913,Fechamento
...,...,...,...,...
245,4.3241,4.3267,2015-12-24 11:04:12.571,Fechamento
246,4.3020,4.3038,2015-12-28 13:10:06.75,Fechamento
247,4.2065,4.2076,2015-12-29 13:11:48.876,Fechamento
248,4.2634,4.2648,2015-12-30 13:05:21.849,Fechamento


In [40]:
df_final['SK_DATA'] = pd.to_datetime(df_final['dataHoraCotacao']).dt.strftime('%Y%m%d')
drop_columns = ['dataHoraCotacao','tipoBoletim']
df_final = df_final.drop(columns=drop_columns)

In [41]:
df_final

,cotacaoCompra,cotacaoVenda,SK_DATA
0,3.2361,3.2379,20150102
1,3.2340,3.2349,20150105
2,3.2165,3.2182,20150106
3,3.1673,3.1683,20150107
4,3.1749,3.1764,20150108
...,...,...,...
245,4.3241,4.3267,20151224
246,4.3020,4.3038,20151228
247,4.2065,4.2076,20151229
248,4.2634,4.2648,20151230


In [42]:
df_compra = pd.DataFrame({'SK_DATA':df_final['SK_DATA'],'INDICADOR':'Euro/Compra',
                          'VALOR_DIA':df_final['cotacaoCompra'],'DTA_CADASTRO':datetime_now})

df_venda = pd.DataFrame({'SK_DATA':df_final['SK_DATA'],'INDICADOR':'Euro/Venda',
                          'VALOR_DIA':df_final['cotacaoVenda'],'DTA_CADASTRO':datetime_now})

df_final = pd.concat([df_compra, df_venda], ignore_index=False)


In [43]:
df_final = df_final.sort_values(by='SK_DATA')
df_final

,SK_DATA,INDICADOR,VALOR_DIA,DTA_CADASTRO
0,20150102,Euro/Compra,3.2361,2024-04-02 11:26:26.710
0,20150102,Euro/Venda,3.2379,2024-04-02 11:26:26.710
1,20150105,Euro/Compra,3.2340,2024-04-02 11:26:26.710
1,20150105,Euro/Venda,3.2349,2024-04-02 11:26:26.710
2,20150106,Euro/Compra,3.2165,2024-04-02 11:26:26.710
...,...,...,...,...
247,20151229,Euro/Compra,4.2065,2024-04-02 11:26:26.710
248,20151230,Euro/Venda,4.2648,2024-04-02 11:26:26.710
248,20151230,Euro/Compra,4.2634,2024-04-02 11:26:26.710
249,20151231,Euro/Compra,4.2482,2024-04-02 11:26:26.710
